# Interview Challenge 5: Streaming Data Processing

## Problem Statement

You need to build a real-time analytics system that processes user clickstream data and detects suspicious activity patterns.

## Requirements

**Data Schema:**
- `user_id` (string)
- `page_url` (string)
- `timestamp` (string in ISO format)
- `ip_address` (string)
- `user_agent` (string)

## Tasks

1. **Real-time Sessionization**: Group events into user sessions (30-minute timeout)
2. **Fraud Detection Rules**:
   - Flag users with >10 page views per minute
   - Flag users accessing from >5 different IP addresses in 5 minutes
   - Flag users with suspicious user agents (e.g., containing 'bot' or 'crawler')
3. **Sliding Window Aggregations**: Calculate rolling statistics over 10-minute windows sliding every 1 minute
4. **Alert Generation**: Generate alerts for flagged suspicious activity

## Technical Requirements
- Use PySpark Structured Streaming
- Implement watermarking for late data handling
- Use appropriate output modes (append, update, complete)
- Handle event time vs processing time
- Include checkpointing for fault tolerance

## Performance Considerations
- Optimize state management
- Configure appropriate trigger intervals
- Handle backpressure gracefully

In [ ]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

# Create Spark session with streaming support
spark = SparkSession.builder \
    .appName("StreamingChallenge") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

# Define schema for clickstream data
clickstream_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("page_url", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("ip_address", StringType(), True),
    StructField("user_agent", StringType(), True)
])

# For demo purposes, we'll simulate streaming data
# In production, this would read from Kafka/S3/EventHubs
def simulate_clickstream():
    """Simulate streaming clickstream data"""
    import random
    from datetime import datetime, timedelta
    
    users = [f"user_{i}" for i in range(1, 11)]
    pages = ["/home", "/products", "/cart", "/checkout", "/profile"]
    
    while True:
        user = random.choice(users)
        page = random.choice(pages)
        timestamp = datetime.now().isoformat()
        ip = f"192.168.1.{random.randint(1, 255)}"
        ua = random.choice([
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
            "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)",
            "bot/1.0 crawler",
            "Mozilla/5.0 (compatible; Googlebot/2.1)"
        ])
        
        yield (user, page, timestamp, ip, ua)
        time.sleep(0.1)  # Simulate real-time data

# Create streaming DataFrame (simulated)
# In real scenario:
# stream_df = spark.readStream \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "localhost:9092") \
#     .option("subscribe", "clickstream") \
#     .load()

# For this challenge, we'll use rate source for simulation
stream_df = spark.readStream \
    .format("rate") \
    .option("rowsPerSecond", 10) \
    .load()

# Transform rate stream to clickstream format
clickstream_df = stream_df.withColumn(
    "user_id", concat(lit("user_"), (col("value") % 10 + 1).cast("string"))
).withColumn(
    "page_url", 
    when((col("value") % 5) == 0, "/home")
    .when((col("value") % 5) == 1, "/products") 
    .when((col("value") % 5) == 2, "/cart")
    .when((col("value") % 5) == 3, "/checkout")
    .otherwise("/profile")
).withColumn(
    "timestamp", to_timestamp(col("timestamp"))
).withColumn(
    "ip_address", concat(lit("192.168.1."), (col("value") % 255 + 1).cast("string"))
).withColumn(
    "user_agent",
    when((col("value") % 4) == 0, "Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
    .when((col("value") % 4) == 1, "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0)")
    .when((col("value") % 4) == 2, "bot/1.0 crawler")
    .otherwise("Mozilla/5.0 (compatible; Googlebot/2.1)")
).drop("value")

# Show initial data structure
print("Clickstream data structure:")
query = clickstream_df.writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", "false") \
    .trigger(processingTime="5 seconds") \
    .start()

# Let it run for a few seconds to see data
import time
time.sleep(10)
query.stop()

# === YOUR SOLUTION GOES HERE ===
# Reinitialize the stream for your solution
clickstream_stream = spark.readStream \
    .format("rate") \
    .option("rowsPerSecond", 10) \
    .load()

# Transform to clickstream format (same as above)
processed_stream = clickstream_stream.withColumn(
    "user_id", concat(lit("user_"), (col("value") % 10 + 1).cast("string"))
).withColumn(
    "page_url", 
    when((col("value") % 5) == 0, "/home")
    .when((col("value") % 5) == 1, "/products") 
    .when((col("value") % 5) == 2, "/cart")
    .when((col("value") % 5) == 3, "/checkout")
    .otherwise("/profile")
).withColumn(
    "timestamp", to_timestamp(col("timestamp"))
).withColumn(
    "ip_address", concat(lit("192.168.1."), (col("value") % 255 + 1).cast("string"))
).withColumn(
    "user_agent",
    when((col("value") % 4) == 0, "Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
    .when((col("value") % 4) == 1, "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0)")
    .when((col("value") % 4) == 2, "bot/1.0 crawler")
    .otherwise("Mozilla/5.0 (compatible; Googlebot/2.1)")
).drop("value")

# Implement the streaming solution:
# 1. Add watermarking
# 2. Sessionization (30-minute sessions)
# 3. Fraud detection rules
# 4. Sliding window aggregations
# 5. Output suspicious activity alerts

print("Implement your streaming solution above!")


## ✅ Solution Reference

**⚠️ Only refer to this after attempting the problem yourself!**

Here's a complete streaming solution implementation:

In [ ]:
# SOLUTION IMPLEMENTATION

# 1. Add watermarking for late data handling
watermarked_stream = processed_stream.withWatermark("timestamp", "10 minutes")

# 2. Sessionization - Group events into 30-minute sessions
from pyspark.sql.window import Window

# Create session windows based on user activity gaps
sessionized_stream = watermarked_stream.withColumn(
    "session_start",
    session_window(col("timestamp"), "30 minutes").start
).withColumn(
    "session_id",
    concat(col("user_id"), lit("_"), date_format("session_start", "yyyyMMddHHmm"))
)

# 3. Fraud Detection Rules

# Rule 1: Users with >10 page views per minute
page_views_per_minute = watermarked_stream.groupBy(
    window("timestamp", "1 minute"),
    "user_id"
).agg(
    count("page_url").alias("page_views")
).filter("page_views > 10")

# Rule 2: Users accessing from >5 different IPs in 5 minutes
ip_spread = watermarked_stream.groupBy(
    window("timestamp", "5 minutes"),
    "user_id"
).agg(
    countDistinct("ip_address").alias("unique_ips")
).filter("unique_ips > 5")

# Rule 3: Suspicious user agents
suspicious_agents = watermarked_stream.filter(
    lower(col("user_agent")).rlike("bot|crawler|spider")
)

# Combine all fraud indicators
fraud_indicators = page_views_per_minute.select(
    "user_id", lit("high_page_views").alias("fraud_type"), "window.start".alias("detected_at")
).union(
    ip_spread.select(
        "user_id", lit("ip_spread").alias("fraud_type"), "window.start".alias("detected_at")
    )
).union(
    suspicious_agents.select(
        "user_id", lit("suspicious_agent").alias("fraud_type"), "timestamp".alias("detected_at")
    )
)

# 4. Sliding Window Aggregations (10-minute windows, slide every 1 minute)
sliding_stats = watermarked_stream.groupBy(
    window("timestamp", "10 minutes", "1 minute"),
    "user_id"
).agg(
    count("page_url").alias("page_views"),
    countDistinct("page_url").alias("unique_pages"),
    countDistinct("ip_address").alias("unique_ips"),
    avg(when(lower(col("user_agent")).rlike("bot|crawler"), 1).otherwise(0)).alias("bot_ratio")
)

# 5. Output alerts for suspicious activity

# Start the fraud detection query
fraud_query = fraud_indicators.writeStream \
    .format("console") \
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/fraud_checkpoint") \
    .trigger(processingTime="30 seconds") \
    .start()

# Start the sliding window stats query
stats_query = sliding_stats.writeStream \
    .format("console") \
    .outputMode("update") \
    .option("checkpointLocation", "/tmp/stats_checkpoint") \
    .trigger(processingTime="30 seconds") \
    .start()

# Start the session tracking query
session_query = sessionized_stream.groupBy(
    "session_id", "user_id"
).agg(
    count("page_url").alias("session_page_views"),
    countDistinct("page_url").alias("session_unique_pages"),
    min("timestamp").alias("session_start"),
    max("timestamp").alias("session_end")
).writeStream \
    .format("console") \
    .outputMode("update") \
    .option("checkpointLocation", "/tmp/session_checkpoint") \
    .trigger(processingTime="30 seconds") \
    .start()

print("🚨 Fraud detection system active!")
print("📊 Monitoring real-time user behavior...")

# In production, you'd run:
# fraud_query.awaitTermination()

# For demo, run for a short time
import time
time.sleep(60)  # Run for 1 minute

# Stop all queries
fraud_query.stop()
stats_query.stop()
session_query.stop()

print("✅ Streaming analysis completed!")
print("\nKey Learnings:")
print("- Watermarking handles late-arriving data")
print("- Sessionization groups user activity over time")
print("- Multiple window types serve different analytics needs")
print("- Checkpointing ensures fault tolerance")
